##### Trend Template
1. The current stock price is above both the 150-day (30-week) and the 200-day (40-week) moving average price lines.
2. The 150-day moving average is above the 200-day moving average.
3. The 200-day moving average line is trending up for at least 1 month (preferably 4–5 months minimum in most cases).
4. The 50-day (10-week) moving average is above both the 150-day and 200-day moving averages.
5. The current stock price is trading above the 50-day moving average.
6. The current stock price is at least 30 percent above its 52-week low. (Many of the best selections will be 100 percent, 300 percent, or greater above their 52-week low before they emerge from a solid consolidation period and mount a large scale advance.)
7. The current stock price is within at least 25 percent of its 52-week high (the closer to a new high the better).
8. The relative strength ranking (as reported in Investor’s Business Daily) is no less than 70, and preferably in the 80s or 90s, which will generally be the case with the better selections.

In [2]:
import baostock as bs
import nbformat as nbf
import pandas as pd
import talib
import time
import random
from tqdm import tqdm

In [3]:
START_DATE = '2010-01-01'

In [4]:
# get all stock code
bs.login()
rs = bs.query_stock_basic()
data_list = []
while rs.error_code == '0' and rs.next():
    data_list.append(rs.get_row_data())
result = pd.DataFrame(data_list, columns=rs.fields)
codes = result.query('type=="1" and status=="1"')['code']
print('Get', len(codes), 'stocks.')

login success!
Get 4077 stocks.
logout success!


In [5]:
candidate = []
for i, code in enumerate(tqdm(codes)):
    rs = bs.query_history_k_data_plus(code, "date,close", start_date=START_DATE, frequency='d', adjustflag='2')
    data_list = []
    while (rs.error_code == '0') & rs.next():
        data_list.append(rs.get_row_data())
    result = pd.DataFrame(data_list, columns=rs.fields)
    close = result['close'].astype('float64')
    sma50 = talib.SMA(close, 50)
    sma150 = talib.SMA(close, 150)
    sma200 = talib.SMA(close, 200)
    delta = 0
    while True:
        if delta + 52*5 >= len(close):
            break
        close52 = close[-delta-52*5:][:52*5]
        close52low = close52.min()
        close52high = close52.max()
        match = False
        if close.iloc[-delta-1] > sma50.iloc[-delta-1] > sma150.iloc[-delta-1] > sma200.iloc[-delta-1]:
            if (sma200.pct_change()[-delta-20:][:20] > 0).all():
                if close.iloc[-delta-1] > close52low * 1.30 and close.iloc[-delta-1] > close52high * 0.75:
                    delta += 1
                    match = True
        if not match:
            break
            
    if delta > 0:
        candidate.append((code, delta, close.iloc[-1], sma50.iloc[-1], sma150.iloc[-1], sma200.iloc[-1], close52low, close52high))
    
    if i % 100 == 0:
        time.sleep(random.random())

bs.logout()

print('Get', len(candidate), 'candidates.')
result = pd.DataFrame(candidate, columns=('code', 'delta', 'close', 'sma50', 'sma150', 'sma200', '52low', '52high'))
# result.sort_values(['delta'])

result['code'].to_csv('stock_list.csv', index=False, header=False)

  0%|          | 0/4077 [00:00<?, ?it/s]

login success!


100%|██████████| 4077/4077 [20:19<00:00,  3.34it/s]  


logout success!
Get 665 candidates.
